In [4]:
# %load_ext autoreload
# %autoreload 2

# import sys
# sys.path.append('..')

# from src import get_segmented_image

In [5]:
# sigma=1
# neighbor=8
# K=5.0
# min_comp_size=3000
# input_file='../img/banana_vehicule.JPG'
# output_file='../img/banana_vehicule2.JPG'

# get_segmented_image(sigma, neighbor, K, min_comp_size, input_file, output_file)

In [6]:
import argparse
import logging
import time
from random import random
from PIL import Image, ImageFilter
from skimage import io
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


In [7]:
class Node:
    def __init__(self, parent, rank=0, size=1):
        self.parent = parent
        self.rank = rank
        self.size = size

    def __repr__(self):
        return '(parent=%s, rank=%s, size=%s)' % (self.parent, self.rank, self.size)

In [8]:
class Forest:
    def __init__(self, num_nodes):
        self.nodes = [Node(i) for i in range(num_nodes)]
        self.num_sets = num_nodes

    def size_of(self, i):
        return self.nodes[i].size

    def find(self, n):
        temp = n
        while temp != self.nodes[temp].parent:
            temp = self.nodes[temp].parent

        self.nodes[n].parent = temp
        return temp

    def merge(self, a, b):
        if self.nodes[a].rank > self.nodes[b].rank:
            self.nodes[b].parent = a
            self.nodes[a].size = self.nodes[a].size + self.nodes[b].size
        else:
            self.nodes[a].parent = b
            self.nodes[b].size = self.nodes[b].size + self.nodes[a].size

            if self.nodes[a].rank == self.nodes[b].rank:
                self.nodes[b].rank = self.nodes[b].rank + 1

        self.num_sets = self.num_sets - 1

    def print_nodes(self):
        for node in self.nodes:
            print(node)

In [9]:
def create_edge(img, width, x, y, x1, y1, diff):
    vertex_id = lambda x, y: y * width + x
    w = diff(img, x, y, x1, y1)
    return (vertex_id(x, y), vertex_id(x1, y1), w)

In [10]:
def build_graph(img, width, height, diff, neighborhood_8=False):
    graph_edges = []
    for y in range(height):
        for x in range(width):
            # If only 4-neighbors are considered, resulting in edges between a pixel and its top and left neighbors.
            if x > 0:
                graph_edges.append(create_edge(img, width, x, y, x-1, y, diff))

            if y > 0:
                graph_edges.append(create_edge(img, width, x, y, x, y-1, diff))

            # 8-neighbors include the 4-neighbors plus diagonally adjacent pixels.
            # If 8-neighbors are considered, resulting in additional edges between a pixel and its diagonal neighbors.
            if neighborhood_8:
                if x > 0 and y > 0:
                    graph_edges.append(create_edge(img, width, x, y, x-1, y-1, diff))

                if x > 0 and y < height-1:
                    graph_edges.append(create_edge(img, width, x, y, x-1, y+1, diff))

    return graph_edges

In [11]:
def remove_small_components(forest, graph, min_size):
    for edge in graph:
        a = forest.find(edge[0])
        b = forest.find(edge[1])

        if a != b and (forest.size_of(a) < min_size or forest.size_of(b) < min_size):
            forest.merge(a, b)

    return  forest

In [12]:
def segment_graph(graph_edges, num_nodes, const, min_size, threshold_func):
    # Step 1: initialization
    forest = Forest(num_nodes)
    weight = lambda edge: edge[2]
    sorted_graph = sorted(graph_edges, key=weight)
    threshold = [ threshold_func(1, const) for _ in range(num_nodes) ]

    # Step 2: merging
    for edge in sorted_graph:
        parent_a = forest.find(edge[0])
        parent_b = forest.find(edge[1])
        a_condition = weight(edge) <= threshold[parent_a]
        b_condition = weight(edge) <= threshold[parent_b]

        if parent_a != parent_b and a_condition and b_condition:
            forest.merge(parent_a, parent_b)
            a = forest.find(parent_a)
            threshold[a] = weight(edge) + threshold_func(forest.nodes[a].size, const)

    return remove_small_components(forest, sorted_graph, min_size)

In [13]:
# It returns the Euclidean distance between the two pixels.

def diff(img, x1, y1, x2, y2):
    _out = np.sum((img[x1, y1] - img[x2, y2]) ** 2)
    return np.sqrt(_out)

In [14]:
def threshold(size, const):
    return (const * 1.0 / size)

In [15]:
def generate_image(forest, width, height):
    random_color = lambda: (int(random()*255), int(random()*255), int(random()*255))
    colors = [random_color() for i in range(width*height)]

    img = Image.new('RGB', (width, height))
    im = img.load()
    for y in range(height):
        for x in range(width):
            comp = forest.find(y * width + x)
            im[x, y] = colors[comp]

    return img.transpose(Image.ROTATE_270).transpose(Image.FLIP_LEFT_RIGHT)

In [16]:
image_path = '../img/banana_vehicule.JPG'

img = Image.open(image_path)

In [17]:
sigma = 1.0
size = img.size

smooth = img.filter(ImageFilter.GaussianBlur(sigma))
smooth = np.array(smooth).astype('int')

In [18]:
graph_edges = build_graph(smooth, size[1], size[0], diff, neighborhood_8=False)

In [19]:
num_nodes = size[0] * size[1]

forest = Forest(num_nodes)

In [20]:
sorted_graph = sorted(graph_edges, key=lambda x: x[2])
threshold = [5 for _ in range(num_nodes)]

#### MaxMin Flow Implementation

In [21]:
from collections import deque
from operator import itemgetter
import sys, getopt, math

import networkx as nx
from networkx.algorithms.flow.utils import build_residual_network

import cv2
import numpy as np
import matplotlib.pyplot as plt

##### BoykovKolmorogov

In [22]:
class BoykovKolmorogov():

    def __init__(self, G, s, t, capacity, resisual=None, cutoff=None):
        self.s = s
        self.t = t
        self.capacity = capacity

        self.St = {s: None}
        self.Tt = {t: None}
        self.A = deque([s, t])
        self.O = deque()
        
        self.flow_value = 0
        self.dist = {s: 0, t: 0}
        self.current_step = 1
        self.steps = {s: 0, t: 0}

        if resisual is None:
            self.residual = build_residual_network(G, capacity)
        else:
            self.residual = resisual

        for p in self.residual:
            for e in self.residual[p].values():
                e['flow'] = 0

        self.inf = self.residual.graph['inf']
        if cutoff is None:
            self.cutoff = self.inf

        self.succ = self.residual.succ
        self.pred = self.residual.pred
        
        
    def grow(self):
        while self.A:
            u = self.A[0]

            if u in self.St:
                current = self.St
                other = self.Tt
                neighbors = self.succ
            else:
                current = self.Tt
                other = self.St
                neighbors = self.pred

            for v, attr in neighbors[u].items():
                if attr['capacity'] > attr['flow']:
                    if v not in current:
                        if v in other:
                            return (u, v) if current is self.St else (v, u)
                        
                        current[v] = u
                        self.dist[v] = self.dist[u] + 1
                        self.steps[v] = self.steps[u]
                        self.A.append(v)

                    elif v in current and self.steps[v] <= self.steps[u] and self.dist[v] > self.dist[u] + 1:
                        current[v] = u
                        self.dist[v] = self.dist[u] + 1
                        self.steps[v] = self.steps[u]

            _ = self.A.popleft()
        return None, None


    def augment(self, u, v):
        attr = self.succ[u][v]
        flow = min(self.inf, attr['capacity'] - attr['flow'])

        # Trace a path from u to s in St
        path = [u]
        w = u
        while w != self.s:
            n = w
            w = self.St[n]
            attr = self.pred[n][w]
            flow = min(flow, attr['capacity'] - attr['flow'])
            path.append(w)
        path.reverse()

        # Trace a path from v to t in Tt
        path.append(v)
        w = v
        while w != self.t:
            n = w
            w = self.Tt[n]
            attr = self.succ[n][w]
            flow = min(flow, attr['capacity'] - attr['flow'])
            path.append(w)

        # Augment flow along the path and check for orphans
        it = iter(path)
        u = next(it)
        new_orphans = []
        for v in it:
            self.succ[u][v]['flow'] += flow
            self.succ[v][u]['flow'] -= flow
            if self.succ[u][v]['flow'] == self.succ[u][v]['capacity']:
                if v in self.St:
                    self.St[v] = None
                    new_orphans.append(v)
                if u in self.Tt:
                    self.Tt[u] = None
                    new_orphans.append(u)
            u = v
        self.O.extend(sorted(new_orphans, key=self.dist.get))
        
        self.flow_value += flow


    def adopt(self):
        while self.O:
            u = self.O.popleft()
            if u in self.St:
                current = self.St
                neighbors = self.pred
            else:
                current = self.Tt
                neighbors = self.succ
            nbrs = ((n, attr, self.dist[n]) for n, attr in neighbors[u].items()
                    if n in current)
            for v, attr, d in sorted(nbrs, key=itemgetter(2)):
                if attr['capacity'] > attr['flow']:
                    if self.valid_root(v, current):
                        current[u] = v
                        self.dist[u] = self.dist[v] + 1
                        self.steps[u] = self.current_step
                        break
            else:
                nbrs = ((n, attr, self.dist[n]) for n, attr in neighbors[u].items()
                        if n in current)
                for v, attr, d in sorted(nbrs, key=itemgetter(2)):
                    if attr['capacity'] > attr['flow']:
                        if v not in self.A:
                            self.A.append(v)
                    if current[v] == u:
                        current[v] = None
                        self.O.appendleft(v)
                if u in self.A:
                    self.A.remove(u)
                del current[u]


    def valid_root(self, n, tree):
        path = []
        v = n
        while v is not None:
            path.append(v)
            if v == self.s or v == self.t:
                base_dist = 0
                break
            elif self.steps[v] == self.current_step:
                base_dist = self.dist[v]
                break
            v = tree[v]
        else:
            return False
        length = len(path)
        for i, u in enumerate(path, 1):
            self.dist[u] = base_dist + length - i
            self.steps[u] = self.current_step
        return True
    

    def max_flow(self):
        while self.flow_value < self.cutoff:
            p, q = self.grow()
            if p is None:
                break
            self.current_step += 1
            self.augment(p, q)
            self.adopt()
        
        self.residual.graph['flow_value'] = self.flow_value
        self.residual.graph['trees'] = (self.St, self.Tt)
        return self.residual

##### Utils

In [23]:
def mark_seeds(event,x,y,flags,param):
    global drawing,mode,marked_bg_pixels,marked_ob_pixels,I_dummy
    h,w,c=I_dummy.shape

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            if mode == "ob":
                if(x>=0 and x<=w-1) and (y>0 and y<=h-1):
                    marked_ob_pixels.append((y,x))
                cv2.line(I_dummy,(x-3,y),(x+3,y),(0,0,255))
            else:
                if(x>=0 and x<=w-1) and (y>0 and y<=h-1):
                    marked_bg_pixels.append((y,x))
                cv2.line(I_dummy,(x-3,y),(x+3,y),(255,0,0))
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if mode == "ob":
            cv2.line(I_dummy,(x-3,y),(x+3,y),(0,0,255))
        else:
            cv2.line(I_dummy,(x-3,y),(x+3,y),(255,0,0))

In [24]:
def draw_sp_mask(I,SP):
	I_marked=np.zeros(I.shape)
	I_marked=np.copy(I)
	mask=SP.getLabelContourMask()
	for i in range(mask.shape[0]):
		for j in range(mask.shape[1]):
			if mask[i][j]==-1 or mask[i][j]==255: # SLIC/SLICO marks borders with -1 :: SEED marks borders with 255
				I_marked[i][j]=[128,128,128]
	return I_marked

In [25]:
def draw_centroids(I, SP_list):
	for each in SP_list:
		i,j=each.centroid
		I[i][j]=128
	return I

In [26]:
def distance(p0, p1):
	return math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2)

##### Img to Graph

In [27]:
class SPNode():
	def __init__(self, label=None, pixels=[], mean_intensity=0.0, centroid=(), type='na', mean_lab=None, lab_hist=None, real_lab=None):
		self.label = label
		self.pixels = pixels
		self.mean_intensity = mean_intensity
		self.centroid = centroid
		self.type = type
		self.mean_lab = mean_lab
		self.lab_hist = lab_hist
		self.real_lab = real_lab
	def __repr__(self):
		return str(self.label)

In [28]:
def gen_sp(image, pixel_obj, pixel_bg, algorithm=101, region_size=20, num_iter=4):
    SP = cv2.ximgproc.createSuperpixelSLIC(image, algorithm=algorithm, region_size=region_size, ruler=10.0)
    SP.iterate(num_iterations=num_iter)

    SP_labels=SP.getLabels()
    SP_list=[None for each in range(SP.getNumberOfSuperpixels())]

    h, w, _ = image.shape
    I_lab=cv2.cvtColor(image,cv2.COLOR_BGR2LAB)

    for i in range(h):
        for j in range(w):
            if not SP_list[SP_labels[i][j]]:
                tmp_sp = SPNode(label=SP_labels[i][j], pixels=[(i,j)])
                SP_list[SP_labels[i][j]]=tmp_sp
            else:
                SP_list[SP_labels[i][j]].pixels.append((i,j))

    for sp in SP_list:
        n_pixels = len(sp.pixels)
        i_sum = 0
        j_sum = 0
        lab_sum = [0,0,0]
        tmp_mask = np.zeros((h,w),np.uint8)
        for each in sp.pixels:
            i,j = each
            i_sum += i
            j_sum += j
            lab_sum = [x + y for x, y in zip(lab_sum, I_lab[i][j])]
            tmp_mask[i][j] = 255
        sp.lab_hist = cv2.calcHist([I_lab], [0,1,2], tmp_mask, (32,32,32), [0, 255, 0, 255, 0, 255])
        sp.centroid += (i_sum//n_pixels, j_sum//n_pixels,)
        sp.mean_lab = [x/n_pixels for x in lab_sum]
        sp.real_lab = [sp.mean_lab[0]*100/255, sp.mean_lab[1]-128, sp.mean_lab[2]-128]

    # Label the marked pixels
    for pixels in pixel_obj:
        x,y = pixels
        SP_list[SP_labels[x][y]].type="ob"
    for pixels in pixel_bg:
        x,y = pixels
        SP_list[SP_labels[x][y]].type="bg"
    mask_ob=np.zeros((h,w),dtype=np.uint8)
    for pixels in pixel_obj:
        i,j=pixels
        mask_ob[i][j]=255
    mask_bg=np.zeros((h,w),dtype=np.uint8)
    for pixels in pixel_bg:
        i,j=pixels
        mask_bg[i][j]=255

    # Get the histograms
    hist_ob=cv2.calcHist([image],[0,1,2],mask_ob,(32,32,32),[0, 255, 0, 255, 0, 255])
    hist_bg=cv2.calcHist([image],[0,1,2],mask_bg,(32,32,32),[0, 255, 0, 255, 0, 255])

    return SP, SP_list, hist_ob, hist_bg   

In [29]:
def gen_graph(I, SP_list, hist_ob, hist_bg, bins=(32,32,32), lambda_=0.9, sigma=5):
    G = nx.Graph()
    s = SPNode(label='s')
    t = SPNode(label='t')
    hist_ob_sum = int(hist_ob.sum())
    hist_bg_sum = int(hist_bg.sum())

    for u in SP_list:
        K=0
        region_rad=math.sqrt(len(u.pixels)/math.pi)
        for v in SP_list:
            if u != v:
                if distance(u.centroid, v.centroid) <= 2.5*region_rad:
                    sim = math.exp(-(cv2.compareHist(u.lab_hist,v.lab_hist,3)**2/2*sigma**2))*(1/distance(u.centroid, v.centroid))
                    K += sim
                    G.add_edge(u, v, sim=sim)
        if(u.type=='na'):
            l_,a_,b_ = [int(x) for x in u.mean_lab]
            l_i = int(l_//(255/bins[0]))
            a_i = int(a_//(255/bins[1]))
            b_i = int(b_//(255/bins[2]))
            pr_ob = int(hist_ob[l_i,a_i,b_i])/hist_ob_sum
            pr_bg = int(hist_bg[l_i,a_i,b_i])/hist_bg_sum
            sim_s = 100000
            sim_t = 100000
            if pr_bg > 0:
                sim_s = lambda_*-np.log(pr_bg)
            if pr_ob > 0:
                sim_t = lambda_*-np.log(pr_ob)
            G.add_edge(s, u, sim=sim_s)
            G.add_edge(t, u, sim=sim_t)
        if(u.type=='ob'):
            G.add_edge(s, u, sim=1+K)
            G.add_edge(t, u, sim=0)
        if(u.type=='bg'):
            G.add_edge(s, u, sim=0)
            G.add_edge(t, u, sim=1+K)		
    return G, s, t

In [30]:
def img_to_graph(image, marked_ob_pixels, marked_bg_pixels, return_sp=True, algorithm=101, region_size=20, num_iter=4, bins=(32,32,32), lambda_=0.9, sigma=5):
    SP, SP_list, hist_ob, hist_bg = gen_sp(image, marked_ob_pixels, marked_bg_pixels, algorithm, region_size, num_iter)
    I_lab=cv2.cvtColor(image,cv2.COLOR_BGR2LAB)
    G, s, t = gen_graph(I_lab, SP_list, hist_ob, hist_bg, bins, lambda_, sigma)

    if return_sp:
        sp_image = np.zeros(image.shape,dtype=np.uint8)
        for sp in SP_list:
            for pixels in sp.pixels:
                i,j=pixels
                sp_image[i][j]=sp.mean_lab
        sp_image=cv2.cvtColor(sp_image, cv2.COLOR_Lab2RGB)
        
        sp_mask=draw_sp_mask(image,SP)
        sp_mask=draw_centroids(sp_mask, SP_list)

        return G, s, t, sp_mask, sp_image

    return G, s, t

##### Implémentation

In [31]:
image_path = '../img/banana_vehicule.JPG'

I = cv2.imread(image_path)
I_dummy=np.zeros(I.shape)
I_dummy=np.copy(I)

In [32]:
cv2.namedWindow('Mark the object and background')
cv2.setMouseCallback('Mark the object and background',mark_seeds)
while(1):
    cv2.imshow('Mark the object and background',I_dummy)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('o'):
        mode = "ob"
    elif k == ord('b'):
        mode = "bg"
    elif k == 27:
        break
cv2.destroyAllWindows()

NameError: name 'drawing' is not defined

NameError: name 'drawing' is not defined

NameError: name 'drawing' is not defined

NameError: name 'drawing' is not defined

NameError: name 'drawing' is not defined

...

NameError: name 'mode' is not defined

NameError: name 'mode' is not defined

NameError: name 'mode' is not defined

KeyboardInterrupt: 

: 